Import statements

In [1]:
import requests
import json
import time

Create initial lists/dictionaries. **Thank you [Master League](https://masterleague.net/) for the data!**

In [ ]:
endpoints = {
    "heroes": "https://api.masterleague.net/heroes/?format=api",
    "maps": "https://api.masterleague.net/maps/?format=api",
    "regions": "https://api.masterleague.net/regions/?format=api",
    "patches": "https://api.masterleague.net/patches/?format=api",
    "teams": "https://api.masterleague.net/teams/?format=api",
    "players": "https://api.masterleague.net/players/?format=api",
    "tournaments": "https://api.masterleague.net/tournaments/?format=api",
    "matches": "https://api.masterleague.net/matches/?format=api",
    "calendar": "https://api.masterleague.net/calendar/?format=api"
}

How to use the API for unauthenticated users. https://api.masterleague.net/?format=api

In [ ]:
#import requests  # http://python-requests.org/

API_ROOT = 'https://api.masterleague.net'
API_USER = None
API_PASS = None

session = requests.Session()

Get data from heroes API.

In [ ]:
r = session.get(API_ROOT + '/heroes.json')
data = json.loads(r.text)
heroes = {}
next_page = data['next']

for hero in data['results']:
    heroes[hero['id']] = hero['name']

while data['next'] is not None:
    next_page = data['next']
    text_results = session.get(next_page)
    data = json.loads(text_results.text)
    
    next_page = data['next']
    for hero in data['results']:
        heroes[hero['id']] = hero['name']

Test call to match API

In [ ]:
match_response = session.get(API_ROOT + '/matches.json')
match_data = json.loads(match_response.text)
match_data

Test call to tournament API

In [ ]:
tournament_response = session.get(API_ROOT + '/tournaments.json')
tournament_data = json.loads(tournament_response.text)
#tournament_data

## Tournament Functions

In [ ]:
def add_tournament_details(tournament_dictionary, page_data):
    for tournament in page_data:
        tournament_id = tournament['id']
        tournament_name = tournament['name']
        tournament_dictionary[tournament_id] = tournament_name
    return tournament_dictionary

def create_tournament_dictionary(api_session, API_ROOT):
    tournaments = {}
    next_tournament_url = API_ROOT + '/tournaments.json'

    while next_tournament_url is not None:
        text_results = api_session.get(next_tournament_url)
        tournament_data = json.loads(text_results.text)
        page_data = tournament_data['results']
        tournaments = add_tournament_details(tournaments, page_data)
        next_tournament_url = tournament_data['next']
    
    return tournaments
    

## Match Functions

In [2]:
def get_match_data(tournament_id, API_ROOT):
    session = requests.Session()
    matches = {}
    match_url = API_ROOT + '/matches.json'
    
    while match_url is not None:
        text_results = session.get(match_url)
        match_data = json.loads(text_results.text)
        #print(match_data)
        for match in match_data['results']:
            tournament = match['tournament']
            if tournament == tournament_id:
                match_id = match['id']
                matches[match_id] = match
        time.sleep(2)
        match_url = match_data['next']
        
    return matches
    

### Create tournament dictionary

In [3]:
API_ROOT = 'https://api.masterleague.net'
API_USER = None
API_PASS = None

session = requests.Session()

In [ ]:
tournament_dict = create_tournament_dictionary(session, API_ROOT)

In [ ]:
tournament_dict

https://api.masterleague.net/?format=api

Get match data for match **32: HGC North America - Phase 1**

In [4]:
match_dict = get_match_data(32, API_ROOT)

KeyError: 'results'

In [ ]:
match_dict

sleep 5 seconds, more than 45 minutes to run. what is the minimum time we can sleep?
Start: 11:46 (sleep 1 second) - wrong. infinite loop.
Restarted: 11:52 (sleep 1 second)